In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow import image

# Commonly used modules
import numpy as np
import os
import sys

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
from google.colab.patches import cv2_imshow as show
import IPython
from six.moves import urllib

print(tf.__version__)

2.8.0


## Load Input Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/My Drive/ENEE439D Project'

 00041000_temp.jpg  'Data folders.gdoc'       KagglePCBData
 00041000_test.jpg   GeneratedData	      PCBData
 00041000.txt	     Image_Processing.ipynb


## Obtain Data Labels

In [ ]:
def one_hot(feature, num_feature):
    code = [0]*num_feature
    code[feature] = 1
    return code

def parse_features(labels_list):
    labels_one_hot = []
    for label in labels_list:
      ohlabel = one_hot(label, 6)
      labels_one_hot.append(ohlabel)
    return labels_one_hot

def load_labels_from_txt(file_path):
    with open(file_path, "r") as f:  # Open file
        data = f.readlines()
        seqs = [line.strip() for line in data]
        print(seqs)
        labels = [seq.split() for seq in seqs]
        print(labels)
        #for each seq, it is a string of digits with spaces. Turn into list.
    
    #max_len = max([len(seq) for seq in seqs])   # Maximum length of the seq
    #seqs, labels = parse_features(seqs, labels, max_len)
    #seqs = np.array(seqs, dtype=np.float32)
    #labels = np.array(labels, dtype=np.float32)
    
    return labels

In [ ]:
labelstxt = '/content/drive/My Drive/ENEE439D Project/GeneratedData/Labels/00041.txt'
labels = load_labels_from_txt(labelstxt)
label_list = [(int(item) - 1) for sublist in labels for item in sublist]
oh_label_list = parse_features(label_list)
print(oh_label_list)

['3 2 4 4 5 6 6 6 1 5', '1 3 4 5 6 3', '2 4 6 6 4 5 6', '1 3 3 5 6 6', '1 2 4 5 6 4 3 3', '1 2 4 6 5 5 6 1 4', '1 4 5 6 5 4 4 3 2', '1 2 3 3 4 5 5 1', '1 2 3 5 6', '1 2 3 4 5 3', '1 4 5 6 4 4 3 2 5', '1 2 2 5 4 5', '1 2 3 5 5', '1 3 5 6 4 3 2 5 4', '1 2 3 3 3 5 6 5 4 5', '1 3 3 5 6', '1 2 3 4 5 6 1 1', '1 3 4 6 5 5 6 3 6', '1 2 3 4 5 6 5', '1 3 5 6 4 3', '1 3 3 4 3 3', '1 4 3 5 6', '1 2 3 5 6', '1 2 3 5 6 4 3 2', '1 2 3 4 5 6 4 3', '1 2 3 4 3 5 6 1', '1 2 3 3 4 3 4 5 6', '1 2 3 6 5 4 3 3 5 6', '1 2 3 4 6 5 4 3 4', '3 5 6', '1 2 3 3 4 6 5 4 5 6', '1 2 3 5 6 5 3 6 2 1', '1 2 3 5 6 5 3 6', '1 2 4 6 5 4', '1 3 6 5 3 4 1 2 4', '1 1 1 4 5 6 5 3 3', '1 2 3 5 6 5 3 4', '1 2 2 3 4 4 6 1', '1 3 3 4 6 5 5', '1 2 3 4 6 5', '1 2 3 3 5 6 5 4 3', '1 1 3 3 6 4 6 5 4 3', '1 1 2 2 3 5 4 6', '1 2 3 6 5 5 4 2 3 1', '1 2 3 4 6 5 3 4', '1 2 3 3 5 6 6 4 4', '1 3 2 4 5 6', '1 2 4 6 5 5 3 4 2 1', '1 2 3 4 5 6 4 3', '1 2 4 4 6 1 2 4', '1 2 3 3 4 6', '1 3 3 2 5 6 3', '1 2 3 3 6 5 4 3 1', '1 2 3 4 5 6 3 1 3', '3 

## Obtain and Convert Context Images

In [ ]:
!ls '/content/drive/My Drive/ENEE439D Project/GeneratedData/Context_Images/00041'

00041000-0.jpg	00041043-2.jpg	00041085-2.jpg	00041125-6.jpg	 00041167-1.jpg
00041000-1.jpg	00041043-3.jpg	00041085-3.jpg	00041125-7.jpg	 00041167-2.jpg
00041000-2.jpg	00041043-4.jpg	00041085-4.jpg	00041125-8.jpg	 00041167-3.jpg
00041000-3.jpg	00041043-5.jpg	00041085-5.jpg	00041126-0.jpg	 00041167-4.jpg
00041000-4.jpg	00041043-6.jpg	00041086-0.jpg	00041126-10.jpg  00041167-5.jpg
00041000-5.jpg	00041043-7.jpg	00041086-1.jpg	00041126-11.jpg  00041167-6.jpg
00041000-6.jpg	00041043-8.jpg	00041086-2.jpg	00041126-1.jpg	 00041168-0.jpg
00041000-7.jpg	00041043-9.jpg	00041086-3.jpg	00041126-2.jpg	 00041168-1.jpg
00041000-8.jpg	00041044-0.jpg	00041086-4.jpg	00041126-3.jpg	 00041168-2.jpg
00041000-9.jpg	00041044-1.jpg	00041086-5.jpg	00041126-4.jpg	 00041168-3.jpg
00041001-0.jpg	00041044-2.jpg	00041086-6.jpg	00041126-5.jpg	 00041168-4.jpg
00041001-1.jpg	00041044-3.jpg	00041086-7.jpg	00041126-6.jpg	 00041168-5.jpg
00041001-2.jpg	00041044-4.jpg	00041086-8.jpg	00041126-7.jpg	 00041168-6.jpg
00041001-3

In [ ]:
import os
FOLDER_PATH = '00041'
ROOT_PATH = '/content/drive/My Drive/ENEE439D Project/GeneratedData/Context_Images/'
joined_path = os.path.join(ROOT_PATH, FOLDER_PATH)
print(len(os.listdir(joined_path)))
image_names = []
for filename in os.listdir(joined_path):
  image_names.append(filename)

1642


In [ ]:
import PIL
from PIL import Image
img_arrays = []
for img_name in image_names:
  image = Image.open(os.path.join(joined_path,img_name))
  img_data = np.asarray(image)
  img_arrays.append(img_data)
# print(image.format)
# print(image.size)
# print(image.mode)
# # show the image
# image.show()
print(img_arrays)

[array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [253, 253, 253],
        [253, 253, 253],
        [253, 253, 253]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]

## Train/Test Data Split

In [ ]:
path = '/content/drive/My Drive/ENEE439D Project/GeneratedData/Context_Images/00041'
train_images = np.array(img_arrays[0:1094]);
preproc_train_images = tf.keras.applications.resnet_v2.preprocess_input(train_images)
test_images = np.array(img_arrays[1094:]);
preproc_test_images = tf.keras.applications.resnet_v2.preprocess_input(test_images)
print(len(preproc_test_images))
train_labels = np.array(oh_label_list[0:1094]);
test_labels = np.array(oh_label_list[1094:]);

548


In [ ]:
test_labels.shape

(548, 6)

## CNN Setup
The ResNet152V2 architecture was used for this.

In [ ]:
from tensorflow.keras.applications import ResNet152V2
model = tf.keras.applications.ResNet152V2(
    include_top=True,
    weights=None,
    pooling='max',
    classes=6,
    classifier_activation="softmax",
)
#model.summary()

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Train on Dataset

In [ ]:
history = model.fit(preproc_train_images, train_labels, epochs=5)

Epoch 1/5
35/35 [==============================] - 77s 2s/step - loss: 2.0149 - accuracy: 0.2130
Epoch 2/5
35/35 [==============================] - 56s 2s/step - loss: 1.8658 - accuracy: 0.2002
Epoch 3/5
35/35 [==============================] - 56s 2s/step - loss: 1.8243 - accuracy: 0.2139
Epoch 4/5
35/35 [==============================] - 56s 2s/step - loss: 1.8208 - accuracy: 0.2102
Epoch 5/5
35/35 [==============================] - 56s 2s/step - loss: 1.8218 - accuracy: 0.2029


## Test Model

In [ ]:
print(test_images.shape)
test_loss, test_acc = model.evaluate(preproc_test_images, test_labels)

print('Test accuracy:', test_acc)

(548, 224, 224, 3)
18/18 [==============================] - 12s 495ms/step - loss: 1.7721 - accuracy: 0.2007
Test accuracy: 0.20072992146015167
